# 🏠 **Analisis Data Aktivitas Marketing & Proyek Perumahan**

## 📘 **Deskripsi Umum**
Dokumen ini berisi kumpulan **analisis SQL komprehensif** yang diterapkan pada database aktivitas marketing, konsumen, proyek, dan wilayah.  
Tujuan utama dari analisis ini adalah untuk:
- Mengevaluasi **performa marketing staff dan supervisor**.  
- Menganalisis **potensi prospek konsumen berdasarkan sumber informasi dan minat pembiayaan**.  
- Menilai **kualitas proyek (blok) berdasarkan skor infrastruktur, bangunan, fasilitas, dan akses lokasi**.  
- Memetakan **aktivitas dan efektivitas marketing pada tiap wilayah (kecamatan)**.

Database terdiri dari enam tabel utama:
1. **aktivitas**
2. **konsumen**
3. **supervisor**
4. **staff_marketing**
5. **proyek**
6. **wilayah**

---

## 🧩 **Struktur Relasi Utama**
Berikut hubungan antar tabel:

- `aktivitas.konsumen_id` → `konsumen.konsumen_id`
- `aktivitas.staff_marketing_id` → `staff_marketing.staff_marketing_id`
- `staff_marketing.supervisor_id` → `supervisor.supervisor_id`
- `aktivitas.blok_id` → `proyek.blok_id`
- `konsumen.kecamatan_id` → `wilayah.kecamatan_id`

---

## 📊 **Daftar Query Analisis**

### 🔹 **Query 1–3: Analisis Dasar Aktivitas dan Prospek**
Analisis awal untuk memahami distribusi data aktivitas dan skor prospek konsumen berdasarkan kategori dasar seperti proyek, referensi informasi, dan minat pembiayaan.

### 🔹 **Query 4–5: Evaluasi Kinerja Marketing dan Supervisor**
Menampilkan performa tiap marketing dan supervisor berdasarkan jumlah prospek, tingkat ketertarikan, serta rata-rata skor prospek.

### 🔹 **Query 6: Rata-rata Skor Prospek berdasarkan Referensi Informasi dan Minat Pembiayaan**
Menilai efektivitas sumber referensi serta minat pembiayaan terhadap skor prospek dan kualitas infrastruktur.

### 🔹 **Query 7: Performa Wilayah (Kecamatan)**
Menganalisis jumlah aktivitas, total prospek tertarik, dan rata-rata skor prospek berdasarkan data wilayah.

### 🔹 **Query 8: Composite Quality Score per Blok**
Menghitung skor komposit kualitas proyek berdasarkan empat aspek utama:
- Kualitas Bangunan (35%)
- Fasilitas (25%)
- Infrastruktur (25%)
- Akses Lokasi (15%)

### 🔹 **Query 9: Staff Terbaik per Blok (Multi-Level CTE)**
Menggunakan teknik **Common Table Expression (CTE)** berlapis untuk menentukan staff terbaik per blok proyek berdasarkan:
- Jumlah aktivitas per blok
- Jumlah prospek tertarik
- Skor rata-rata prospek
- Kualitas bangunan proyek  
Hasil akhirnya menampilkan **ranking staff terbaik di tiap blok** berdasarkan skor gabungan (*staff_block_score*).

---

## 🧠 **Catatan Teknis**
Seluruh query dieksekusi menggunakan Python dengan modul `pandas` dan `sqlalchemy`:

```python
df = pd.read_sql(query, engine)
df

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine


# Path file Excel
file_path = r"S:\2. Kursus dan Pembelajaran\0. Data Analytics Project\Project 7 - Marketing Activity and Prospects\Database dan Jupyter\sql and python\database_activity_kbg.xlsx"

# Membaca semua sheet sekaligus
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Assign dataframe untuk setiap sheet
aktivitas = all_sheets.get("aktivitas")
konsumen = all_sheets.get("konsumen")
staff_marketing = all_sheets.get("staff_marketing")
supervisor = all_sheets.get("supervisor")
proyek = all_sheets.get("proyek")
wilayah = all_sheets.get("wilayah")

# Cek isi awal dari setiap dataframe
print("aktivitas:")
print(aktivitas.head(3), "\n")

print("konsumen:")
print(konsumen.head(3), "\n")

print("staff_marketing:")
print(staff_marketing.head(3), "\n")

print("supervisor:")
print(supervisor.head(3), "\n")

print("proyek:")
print(proyek.head(3), "\n")

print("wilayah:")
print(wilayah.head(3), "\n")

aktivitas:
   activity_id    tanggal  staff_marketing_id  supervisor_id  konsumen_id  \
0            1 2018-01-07                   1              1          733   
1            2 2018-01-13                   1              1           36   
2            3 2018-01-20                   2              1         1177   

  referensi_informasi       prospek_konsumen  skor_prospek_konsumen  \
0        Iklan Tiktok  Tanya Jawab Informasi                      2   
1    Banner Pemasaran    Perencanaan Booking                     10   
2    Brosur Pemasaran  Tanya Jawab Informasi                      2   

      minat_proyek  blok_id       minat_pembiayaan  komunikasi_marketing  \
0  Green Watubelah       12  Cash Bertahap 3 Bulan              8.278636   
1  Green Watubelah       57  Cash Bertahap 6 Bulan              7.229604   
2  Green Watubelah       38  Cash Bertahap 3 Bulan              6.568630   

   kepercayaan_marketing  responsif_marketing  pemahaman_produk_marketing  \
0            

In [13]:
# ===============================
# 3️⃣ Buat koneksi ke SQLite (in-memory)
# ===============================
engine = create_engine('sqlite://', echo=False)

# Masukkan semua sheet ke database
for sheet_name, df in all_sheets.items():
    df.to_sql(sheet_name, con=engine, index=False, if_exists='replace')

print("\n✅ Semua sheet sudah dimuat ke database SQLite!\n")


✅ Semua sheet sudah dimuat ke database SQLite!



## 📊 **Query 1: Analisis KPI (Key Performance Indicator) per Staff Marketing**

### **🎯 Tujuan:**  
Mengukur kinerja setiap staff marketing berdasarkan aktivitas mereka dalam berinteraksi dengan konsumen, tingkat ketertarikan konsumen, dan berbagai metrik skor penilaian dari aktivitas marketing.

In [14]:
query = """

-- Query 1: KPI per staff marketing
WITH staff_agg AS (
  SELECT
    s.staff_marketing_id,
    s.nama_marketing,
    COUNT(a.activity_id) AS total_aktivitas,
    SUM(CASE WHEN a.prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END) AS total_tertarik,
    ROUND(AVG(a.skor_prospek_konsumen), 2) AS avg_skor_prospek,
    ROUND(AVG(a.skor_kualitas_bangunan), 2) AS avg_skor_kualitas_bangunan,
    ROUND(AVG(a.komunikasi_marketing), 2) AS avg_komunikasi
  FROM staff_marketing s
  LEFT JOIN aktivitas a ON s.staff_marketing_id = a.staff_marketing_id
  GROUP BY s.staff_marketing_id, s.nama_marketing
)
SELECT
  staff_marketing_id,
  nama_marketing,
  total_aktivitas,
  total_tertarik,
  total_aktivitas,
  avg_skor_prospek,
  avg_skor_kualitas_bangunan,
  avg_komunikasi,
  ROUND(CASE WHEN total_aktivitas = 0 THEN 0 ELSE (CAST(total_tertarik AS REAL)/total_aktivitas)*100 END,2) AS konversi_persen
FROM staff_agg
ORDER BY konversi_persen DESC, total_aktivitas DESC;


"""

df = pd.read_sql(query, engine)
df

,staff_marketing_id,nama_marketing,total_aktivitas,total_tertarik,total_aktivitas,avg_skor_prospek,avg_skor_kualitas_bangunan,avg_komunikasi,konversi_persen
0,249,Asep Azis Saprizal,379,0,379,4.52,7.43,8.98,0.0
1,246,Farah Nida Aliyah,327,0,327,4.75,7.81,9.04,0.0
2,238,Yeni Aprilia,293,0,293,4.09,7.46,8.97,0.0
3,250,Susilawati,236,0,236,4.22,7.46,9.01,0.0
4,243,Fathan Dhony Fadil,223,0,223,4.47,7.67,8.99,0.0
...,...,...,...,...,...,...,...,...,...
350,351,Pangeran Juliansyah Putra,4,0,4,6.00,5.38,6.28,0.0
351,352,Ari Gunawan,4,0,4,6.00,7.73,5.77,0.0
352,132,Rahmat Cahya Rahman,3,0,3,2.00,8.53,6.71,0.0
353,353,Lolita Putri Santiniyah,3,0,3,4.67,5.70,5.98,0.0


### **🧠 Penjelasan:**  
Query ini menggunakan konsep **CTE (Common Table Expression)** bernama `staff_agg` untuk menghitung metrik agregasi setiap marketing, meliputi:
- Jumlah total aktivitas (`COUNT(a.activity_id)`)
- Jumlah prospek yang tertarik (`SUM(CASE WHEN a.prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END)`)
- Rata-rata skor prospek, kualitas bangunan, dan komunikasi marketing (`AVG()` dengan `ROUND()` untuk pembulatan dua desimal)

### **📈 Kesimpulan:**  
Query ini memberikan gambaran jelas tentang efektivitas setiap staff marketing dalam menarik minat konsumen.  
Dari hasilnya, kita dapat:
- Mengidentifikasi siapa staff dengan konversi prospek tertinggi.  
- Melihat siapa yang paling aktif berinteraksi dengan konsumen.  
- Menentukan prioritas pelatihan atau penghargaan berdasarkan KPI aktual.

## 🧭 **Query 2 : KPI Supervisor dengan Ranking Internal**

### 🎯 **Tujuan**
Menilai performa masing-masing **supervisor** berdasarkan aktivitas tim marketing yang mereka bina.  
Analisis ini bertujuan untuk mengukur efektivitas dan kualitas pembinaan supervisor melalui **jumlah aktivitas, tingkat prospek tertarik, serta skor rata-rata prospek**.  
Hasil akhirnya berupa **peringkat (ranking)** supervisor berdasarkan rata-rata skor prospek tertinggi.

In [15]:
query = """

-- Query 2: KPI per supervisor dengan ranking internal
WITH sup_agg AS (
  SELECT
    sup.supervisor_id,
    sup.nama_supervisor,
    COUNT(a.activity_id) AS total_aktivitas,
    SUM(CASE WHEN a.prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END) AS total_tertarik,
    ROUND(AVG(a.skor_prospek_konsumen),2) AS avg_skor_prospek
  FROM supervisor sup
  LEFT JOIN staff_marketing s ON s.supervisor_id = sup.supervisor_id
  LEFT JOIN aktivitas a ON a.staff_marketing_id = s.staff_marketing_id
  GROUP BY sup.supervisor_id, sup.nama_supervisor
)
SELECT
  supervisor_id,
  nama_supervisor,
  total_aktivitas,
  total_tertarik,
  avg_skor_prospek,
  ROUND(CASE WHEN total_aktivitas = 0 THEN 0 ELSE (CAST(total_tertarik AS REAL)/total_aktivitas)*100 END,2) AS konversi_persen,
  RANK() OVER (ORDER BY avg_skor_prospek DESC) AS rank_by_skor
FROM sup_agg
ORDER BY rank_by_skor;

"""

df = pd.read_sql(query, engine)
df

,supervisor_id,nama_supervisor,total_aktivitas,total_tertarik,avg_skor_prospek,konversi_persen,rank_by_skor
0,15,Dewi Warianti Aprilia,744,0,4.39,0.0,1
1,10,Reny Anggraeni,1331,0,4.37,0.0,2
2,12,Muhammad Apandi,627,0,4.36,0.0,3
3,14,Nurlaela,1505,0,4.31,0.0,4
4,11,Kartika Dewi,865,0,4.27,0.0,5
5,2,Deny Romdhony,384,0,4.26,0.0,6
6,13,Muhammad Syarifudin,739,0,4.23,0.0,7
7,8,Siska Sherlita,880,0,4.08,0.0,8
8,9,Raihan Sulthan Abrar,670,0,4.05,0.0,9
9,3,Nur Indah Surma,225,0,4.04,0.0,10


### 📈 **Interpretasi Hasil**
Output dari query ini akan menampilkan:
- **Total Aktivitas:** seberapa aktif tim di bawah supervisor tersebut.
- **Total Prospek Tertarik:** jumlah calon konsumen yang menunjukkan ketertarikan.
- **Rata-rata Skor Prospek:** indikator kualitas hasil interaksi marketing.
- **Konversi Persen:** efektivitas supervisor dalam mengonversi aktivitas menjadi prospek.
- **Ranking:** urutan supervisor berdasarkan skor prospek tertinggi.

Supervisor dengan peringkat terbaik menunjukkan:
- Kemampuan koordinasi dan pembinaan tim yang efektif.
- Efisiensi dalam membangun ketertarikan konsumen.
- Potensi menjadi benchmark bagi supervisor lain dalam evaluasi kinerja.

### ✅ **Kesimpulan**
Analisis KPI ini memberikan gambaran menyeluruh tentang **efektivitas pembinaan supervisor**.  
Dari hasilnya, perusahaan dapat:
- Mengidentifikasi supervisor dengan performa unggul.
- Menentukan kebutuhan **pelatihan atau coaching tambahan** bagi supervisor dengan nilai rendah.
- Menggunakan hasil ranking sebagai dasar **evaluasi tahunan** atau **bonus kinerja**.

## 🏗️ **Query 3: Top Proyek Berdasarkan Jumlah Leads Berkualitas**

### 🎯 **Tujuan**
Menentukan **proyek-proyek terbaik** berdasarkan **jumlah dan kualitas prospek (leads)** yang dihasilkan oleh aktivitas marketing.  
Analisis ini membantu mengidentifikasi proyek mana yang paling diminati oleh calon konsumen dan memiliki persepsi positif terhadap kualitas infrastruktur serta progres pembangunan.


In [16]:
query = """

-- Query 3: Top proyek berdasarkan jumlah leads berkualitas
SELECT
  p.blok_id,
  p.nama_proyek,
  p.blok,
  COUNT(a.activity_id) AS jumlah_leads,
  SUM(CASE WHEN a.skor_prospek_konsumen >= 7 THEN 1 ELSE 0 END) AS leads_berkualitas,
  ROUND(AVG(a.skor_prospek_konsumen),2) AS avg_skor_prospek,
  ROUND(AVG(a.skor_kualitas_infrastruktur),2) AS avg_kualitas_infrastruktur,
  p.progress_pembangunan
FROM proyek p
LEFT JOIN aktivitas a ON p.blok_id = a.blok_id
GROUP BY p.blok_id, p.nama_proyek, p.blok, p.progress_pembangunan
HAVING COUNT(a.activity_id) > 5
ORDER BY leads_berkualitas DESC, avg_skor_prospek DESC
LIMIT 10;


"""

df = pd.read_sql(query, engine)
df

,blok_id,nama_proyek,blok,jumlah_leads,leads_berkualitas,avg_skor_prospek,avg_kualitas_infrastruktur,progress_pembangunan
0,1005,Taman Banjarwangunan 3,A,13,5,6.62,7.91,Telah Selesai Dibangun
1,594,Taman Banjarwangunan 1,B,6,4,7.67,8.30,Telah Selesai Dibangun
2,90,Green Astapada,A,9,4,6.67,7.35,Telah Selesai Dibangun
3,246,Taman Setu Permai I,D,6,3,7.00,7.13,Telah Selesai Dibangun
4,994,Taman Banjarwangunan 3,A,6,3,7.00,7.81,Telah Selesai Dibangun
5,2668,Gebang Permai,H,6,3,7.00,6.57,Proses Pembangunan Rumah
6,126,Green Astapada,A,7,3,6.57,7.30,Telah Selesai Dibangun
7,1071,Taman Banjarwangunan 3,B,8,3,6.50,7.54,Telah Selesai Dibangun
8,505,Griya Pesona Dawuan,A,6,3,6.33,7.03,Telah Selesai Dibangun
9,2197,Graha Amelia Pabedilan,A,7,3,6.29,7.11,Kavling Tersedia


### 📊 **Interpretasi Hasil**
Output menampilkan:
- **Nama Proyek dan Blok:** identitas proyek yang dievaluasi.
- **Jumlah Leads:** total aktivitas yang menunjukkan minat calon konsumen.
- **Leads Berkualitas:** jumlah prospek dengan skor ≥ 7, menunjukkan ketertarikan tinggi.
- **Rata-rata Skor Prospek dan Infrastruktur:** menilai kualitas persepsi konsumen terhadap proyek.
- **Progress Pembangunan:** tahap kemajuan proyek saat ini.

Proyek dengan **jumlah leads berkualitas tinggi** dan **rata-rata skor tinggi** menandakan:
- Tingginya daya tarik proyek di mata calon pembeli.
- Persepsi positif terhadap kualitas fisik maupun infrastruktur proyek.
- Potensi besar untuk menjadi prioritas pemasaran dan pengembangan.

### ✅ **Kesimpulan**
Analisis ini menyoroti **10 proyek teratas** yang paling efektif menarik minat konsumen berkualitas.  
Dari hasilnya, perusahaan dapat:
- Menentukan **prioritas investasi dan promosi** pada proyek yang paling diminati.
- Mengevaluasi proyek dengan **skor rendah** untuk perbaikan infrastruktur atau strategi marketing.
- Menggunakan hasil ini untuk **perencanaan strategi penjualan dan pengalokasian sumber daya marketing** secara lebih tepat sasaran

## 👥 **Query 4: Segmentasi Konsumen Berdasarkan Potensi Nilai dan Skor Prospek**

### 🎯 **Tujuan**
Membuat **segmentasi konsumen** berdasarkan kombinasi **tingkat pendapatan (penghasilan)** dan **skor prospek terakhir** yang menunjukkan minat terhadap proyek.  
Tujuan utama analisis ini adalah untuk **mengidentifikasi prioritas calon pembeli** — siapa yang termasuk kategori *High Value*, *Medium Value*, atau *Low Value* — sehingga strategi pemasaran dapat diarahkan lebih efektif.

In [17]:
query = """

-- Query 4: Segmentasi konsumen
WITH konsumen_latest AS (
  -- Ambil aktivitas terakhir per konsumen
  SELECT
    k.konsumen_id,
    k.nama_lengkap,
    k.jenis_kelamin,
    k.usia,
    k.penghasilan,
    a.skor_prospek_konsumen,
    a.tanggal
  FROM konsumen k
  LEFT JOIN aktivitas a ON k.konsumen_id = a.konsumen_id
  WHERE a.tanggal IS NOT NULL
  AND a.tanggal = (
    SELECT MAX(t2.tanggal) FROM aktivitas t2 WHERE t2.konsumen_id = k.konsumen_id
  )
)
SELECT
  konsumen_id,
  nama_lengkap,
  usia,
  penghasilan,
  skor_prospek_konsumen,
  CASE
    WHEN skor_prospek_konsumen >= 8 AND penghasilan >= 8000000 THEN 'High Value'
    WHEN skor_prospek_konsumen >= 6 AND penghasilan >= 5000000 THEN 'Medium Value'
    ELSE 'Low Value'
  END AS segment_prospek
FROM konsumen_latest
ORDER BY skor_prospek_konsumen DESC, penghasilan DESC
LIMIT 100;


"""

df = pd.read_sql(query, engine)
df

,konsumen_id,nama_lengkap,usia,penghasilan,skor_prospek_konsumen,segment_prospek
0,1831,Rahayu Wardani Butarbutar,29,7400000,10,Medium Value
1,1021,Fajar Fahmi Kaban,42,6970000,10,Medium Value
2,567,Utomo Hudaya Kusnadi Wibisono,34,6690000,10,Medium Value
3,749,Purnomo Arif Nasution,36,6650000,10,Medium Value
4,435,Dian Lumban,25,6640000,10,Medium Value
...,...,...,...,...,...,...
95,1155,Taufik Situmorang,48,5310000,10,Medium Value
96,718,Yoga Mahardika,45,5300000,10,Medium Value
97,1326,Bambang Harefa,33,5300000,10,Medium Value
98,340,Sari Purnamasari Ginting,26,5270000,10,Medium Value


### 📊 **Interpretasi Hasil**
Output menampilkan:
- **Nama dan usia konsumen** → identitas dasar calon pembeli.
- **Penghasilan dan skor prospek terakhir** → kombinasi minat dan daya beli.
- **Segmentasi konsumen (`segment_prospek`)** → klasifikasi strategis untuk pemasaran.

Interpretasinya:
- **High Value:** calon konsumen dengan minat tinggi dan kemampuan finansial kuat → prioritas utama follow-up.  
- **Medium Value:** memiliki potensi baik, bisa ditingkatkan dengan pendekatan promosi atau pembiayaan yang sesuai.  
- **Low Value:** minat rendah atau daya beli terbatas, cocok untuk strategi jangka panjang.

### ✅ **Kesimpulan**
Segmentasi konsumen berbasis data ini memungkinkan tim marketing untuk:
- **Memfokuskan sumber daya** pada konsumen bernilai tinggi.
- **Menyesuaikan strategi komunikasi dan penawaran** berdasarkan segmen.
- **Meningkatkan efisiensi dan efektivitas kampanye pemasaran.**

Dengan memanfaatkan data aktivitas terakhir dan profil keuangan, perusahaan dapat lebih akurat dalam menentukan siapa yang berpotensi menjadi pembeli aktual dan siapa yang masih memerlukan nurturing lebih lanjut.

## 📅 **Query 5: Analisis Funnel Aktivitas Marketing per Bulan**

### 🎯 **Tujuan**
Menganalisis performa aktivitas marketing secara **bulanan (monthly funnel)** untuk melihat **tren jumlah aktivitas, prospek tertarik, dan tingkat konversi** dari waktu ke waktu.  
Tujuan utama adalah memahami pola musim, peningkatan, atau penurunan performa marketing dari bulan ke bulan.

In [18]:
query = """

-- Query 5: Monthly funnel
WITH monthly AS (
  SELECT
    strftime('%Y-%m', tanggal) AS year_month,
    COUNT(activity_id) AS total_aktivitas,
    SUM(CASE WHEN prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END) AS total_tertarik,
    ROUND(AVG(skor_prospek_konsumen),2) AS avg_skor_prospek
  FROM aktivitas
  WHERE tanggal IS NOT NULL
  GROUP BY year_month
)
SELECT
  year_month,
  total_aktivitas,
  total_tertarik,
  avg_skor_prospek,
  ROUND(CASE WHEN total_aktivitas = 0 THEN 0 ELSE (CAST(total_tertarik AS REAL)/total_aktivitas)*100 END,2) AS konversi_persen
FROM monthly
ORDER BY year_month DESC;


"""

df = pd.read_sql(query, engine)
df

,year_month,total_aktivitas,total_tertarik,avg_skor_prospek,konversi_persen
0,2025-10,127,0,4.98,0.0
1,2025-09,186,0,4.26,0.0
2,2025-08,169,0,4.50,0.0
3,2025-07,161,0,3.96,0.0
4,2025-06,218,0,4.57,0.0
...,...,...,...,...,...
89,2018-05,12,0,3.67,0.0
90,2018-04,11,0,4.00,0.0
91,2018-03,9,0,3.33,0.0
92,2018-02,12,0,4.67,0.0


### 📊 **Interpretasi Hasil**
Output menampilkan:
- **Year-Month:** periode analisis.
- **Total Aktivitas:** jumlah kegiatan marketing yang tercatat.
- **Total Tertarik:** jumlah prospek dengan status *tertarik*.
- **Rata-rata Skor Prospek:** menunjukkan kualitas rata-rata minat konsumen.
- **Konversi Persen:** persentase aktivitas yang berhasil menghasilkan prospek tertarik.

Melalui hasil ini, tim marketing dapat:
- Mengamati tren peningkatan atau penurunan jumlah prospek tertarik per bulan.
- Menilai apakah kenaikan aktivitas sejalan dengan peningkatan konversi.
- Menentukan waktu atau bulan dengan performa terbaik untuk dijadikan acuan strategi berikutnya.


### ✅ **Kesimpulan**
Analisis funnel bulanan membantu perusahaan:
- **Memantau performa tim marketing secara periodik.**
- **Mengidentifikasi bulan-bulan dengan efektivitas tinggi atau rendah.**
- **Mengambil keputusan berbasis data** untuk pengaturan target dan strategi promosi di masa mendatang.

Dengan demikian, metrik *monthly funnel* menjadi dasar penting dalam pelaporan kinerja marketing dan perencanaan kampanye berkelanjutan.

## 🔍 **Query 6: Analisis Rata-Rata Skor Prospek Berdasarkan Sumber Referensi dan Minat Pembiayaan**

### 🎯 **Tujuan**
Menganalisis **efektivitas sumber referensi informasi** (misalnya media sosial, brosur, teman, pameran, dll.) serta **jenis minat pembiayaan** (misalnya KPR bank, tunai, in-house, dll.) terhadap **kualitas prospek dan persepsi infrastruktur proyek**.  
Tujuan utama adalah untuk memahami **sumber informasi mana** yang menghasilkan prospek paling berkualitas dan **metode pembiayaan mana** yang paling diminati oleh calon konsumen potensial.

In [19]:
query = """

-- Query 6: Rata-rata skor prospek by sumber referensi dan minat pembiayaan
SELECT
  a.referensi_informasi,
  a.minat_pembiayaan,
  COUNT(a.activity_id) AS jumlah_aktivitas,
  ROUND(AVG(a.skor_prospek_konsumen),2) AS avg_skor_prospek,
  ROUND(AVG(a.skor_kualitas_infrastruktur),2) AS avg_kualitas_infrastruktur
FROM aktivitas a
GROUP BY a.referensi_informasi, a.minat_pembiayaan
HAVING COUNT(a.activity_id) >= 10
ORDER BY avg_skor_prospek DESC;

"""

df = pd.read_sql(query, engine)
df

,referensi_informasi,minat_pembiayaan,jumlah_aktivitas,avg_skor_prospek,avg_kualitas_infrastruktur
0,Youtube,Cash Keras 1 Bulan,22,5.55,7.46
1,Brosur Pemasaran,Cash Bertahap 3 Bulan,69,4.87,7.56
2,Brosur Pemasaran,Cash Bertahap 1 Tahun,96,4.79,7.91
3,Brosur Pemasaran,Take Over Kredit,59,4.68,7.21
4,Youtube,Cash Bertahap 6 Bulan,63,4.67,7.61
5,Youtube,Cash Bertahap 3 Bulan,35,4.63,7.85
6,What'sApp Business,Take Over Kredit,76,4.58,7.28
7,Youtube,Take Over Kredit,50,4.56,7.66
8,Linkedin,Cash Bertahap 3 Bulan,41,4.54,7.59
9,Iklan Instagram,Cash Bertahap 3 Bulan,121,4.50,7.66


### 📊 **Interpretasi Hasil**
Output akan menampilkan:
- **Referensi Informasi:** sumber utama konsumen mengetahui proyek.
- **Minat Pembiayaan:** preferensi pembelian properti.
- **Jumlah Aktivitas:** volume interaksi dari kombinasi tersebut.
- **Rata-Rata Skor Prospek:** kualitas calon pembeli yang dihasilkan dari kanal tersebut.
- **Rata-Rata Skor Infrastruktur:** persepsi umum terhadap kualitas proyek dari kelompok konsumen terkait.

Interpretasinya:
- Nilai **avg_skor_prospek tinggi** menunjukkan kanal referensi dan tipe pembiayaan yang efektif menarik calon pembeli serius.
- Jika **avg_kualitas_infrastruktur juga tinggi**, artinya konsumen dari kanal tersebut juga memiliki persepsi positif terhadap proyek yang dipromosikan.


### ✅**Kesimpulan**
Analisis ini memberikan wawasan strategis bagi tim marketing untuk:
- **Mengidentifikasi kanal promosi paling efektif** berdasarkan kualitas prospek yang dihasilkan.
- **Menentukan fokus pembiayaan** yang paling diminati oleh calon konsumen potensial.
- **Mengoptimalkan anggaran promosi dan komunikasi pemasaran** pada kombinasi referensi–pembiayaan dengan kinerja terbaik.

Dengan demikian, hasil ini dapat digunakan untuk **peningkatan efektivitas kampanye pemasaran dan strategi penjualan** yang lebih terarah dan berbasis data.

## 🗺️ **Query 7: Analisis Performa Marketing Berdasarkan Kecamatan (Wilayah)**

### 🎯 **Tujuan**
Menilai **performa aktivitas marketing dan potensi konsumen berdasarkan wilayah (kecamatan)**.  
Tujuan utama dari analisis ini adalah untuk mengidentifikasi daerah dengan **aktivitas pemasaran paling tinggi**, **jumlah prospek tertarik terbesar**, serta **rata-rata skor prospek tertinggi**, agar strategi pemasaran bisa lebih tepat sasaran secara geografis.

In [20]:
query = """

-- Query 7: Performa by kecamatan (wilayah)
SELECT
  w.kecamatan_id,
  w.nama_kecamatan,
  COUNT(a.activity_id) AS jumlah_aktivitas,
  SUM(CASE WHEN a.prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END) AS total_tertarik,
  ROUND(AVG(a.skor_prospek_konsumen),2) AS avg_skor_prospek,
  w.jumlah_penduduk
FROM wilayah w
LEFT JOIN konsumen k ON k.kecamatan_id = w.kecamatan_id
LEFT JOIN aktivitas a ON a.konsumen_id = k.konsumen_id
GROUP BY w.kecamatan_id, w.nama_kecamatan, w.jumlah_penduduk
ORDER BY total_tertarik DESC
LIMIT 20;


"""

df = pd.read_sql(query, engine)
df

,kecamatan_id,nama_kecamatan,jumlah_aktivitas,total_tertarik,avg_skor_prospek,jumlah_penduduk
0,1,Harjamukti,417,0,4.42,128778
1,2,Kejaksan,365,0,3.97,46670
2,3,Kesambi,120,0,4.27,78398
3,4,Lemahwungkuk,241,0,3.99,58418
4,5,Pekalipan,199,0,4.57,29716
5,6,Astanajapura,390,0,4.31,83200
6,7,Babakan,168,0,4.30,77200
7,8,Ciledug,216,0,4.55,48100
8,9,Depok,273,0,3.95,72300
9,11,Dukupuntang,136,0,4.31,68300


### 📊 **Interpretasi Hasil**
Output menampilkan:
- **Nama Kecamatan** → lokasi asal konsumen.
- **Jumlah Aktivitas** → seberapa banyak kegiatan marketing tercatat di wilayah tersebut.
- **Total Prospek Tertarik** → jumlah calon pembeli potensial dari wilayah itu.
- **Rata-Rata Skor Prospek** → kualitas atau tingkat minat konsumen dari daerah tersebut.
- **Jumlah Penduduk** → konteks populasi untuk membandingkan potensi pasar relatif antar wilayah.

Interpretasinya:
- Kecamatan dengan **jumlah tertarik tinggi dan skor prospek tinggi** merupakan **wilayah potensial** untuk ekspansi pemasaran.
- Jika **jumlah aktivitas rendah tetapi skor tinggi**, wilayah tersebut bisa menjadi target prioritas untuk **peningkatan intensitas marketing**.
- Sebaliknya, wilayah dengan aktivitas banyak tetapi skor prospek rendah bisa menunjukkan **efektivitas promosi yang belum optimal**.

### ✅ **Kesimpulan**
Analisis berdasarkan kecamatan memberikan pandangan spasial terhadap efektivitas aktivitas marketing.  
Dari hasil ini, perusahaan dapat:
- **Menentukan wilayah prioritas pemasaran dan promosi.**
- **Mengalokasikan sumber daya marketing secara lebih efisien.**
- **Menyesuaikan strategi komunikasi** berdasarkan karakteristik dan potensi wilayah tertentu.

Dengan demikian, analisis performa per wilayah ini menjadi dasar penting untuk **pengambilan keputusan berbasis lokasi (location-based marketing strategy)**.

## 🧩 **Query 8: Analisis Skor Kualitas Gabungan (Composite Quality Score) per Blok Proyek**

### 🎯 **Tujuan**
Mengukur **tingkat kualitas proyek perumahan (blok)** berdasarkan berbagai aspek yang dinilai oleh konsumen melalui aktivitas marketing.  
Tujuan utama dari analisis ini adalah untuk **menilai kualitas keseluruhan proyek** secara objektif dan menemukan **blok proyek terbaik** berdasarkan skor gabungan dari beberapa indikator kualitas.

In [21]:
query = """

-- Query 8: Composite quality score per blok
WITH block_scores AS (
  SELECT
    p.blok_id,
    p.nama_proyek,
    COUNT(a.activity_id) AS jumlah_aktivitas,
    ROUND(AVG(a.skor_kualitas_bangunan),2) AS avg_kualitas_bangunan,
    ROUND(AVG(a.skor_fasilitas_bangunan),2) AS avg_fasilitas,
    ROUND(AVG(a.skor_kualitas_infrastruktur),2) AS avg_infrastruktur,
    ROUND(AVG(a.skor_akses_lokasi),2) AS avg_akses
  FROM proyek p
  LEFT JOIN aktivitas a ON p.blok_id = a.blok_id
  GROUP BY p.blok_id, p.nama_proyek
)
SELECT
  blok_id,
  nama_proyek,
  jumlah_aktivitas,
  avg_kualitas_bangunan,
  avg_fasilitas,
  avg_infrastruktur,
  avg_akses,
  ROUND(
    COALESCE(avg_kualitas_bangunan,0)*0.35 +
    COALESCE(avg_fasilitas,0)*0.25 +
    COALESCE(avg_infrastruktur,0)*0.25 +
    COALESCE(avg_akses,0)*0.15
  ,2) AS composite_quality_score
FROM block_scores
WHERE jumlah_aktivitas > 3
ORDER BY composite_quality_score DESC
LIMIT 20;


"""

df = pd.read_sql(query, engine)
df

,blok_id,nama_proyek,jumlah_aktivitas,avg_kualitas_bangunan,avg_fasilitas,avg_infrastruktur,avg_akses,composite_quality_score
0,1035,Taman Banjarwangunan 3,4,9.24,8.27,8.87,9.07,8.88
1,1726,Griya Jatinunggal,5,9.09,7.11,9.55,9.87,8.83
2,1883,Graha Amelia Nanggela,4,9.89,8.16,7.98,8.60,8.79
3,1483,Griya Jatinunggal,5,9.11,8.91,8.04,8.67,8.73
4,1281,Griya Jatinunggal,4,9.23,7.85,8.69,8.75,8.68
5,1734,Griya Jatinunggal,5,8.40,9.94,8.69,7.21,8.68
6,2569,Gebang Permai,4,9.79,7.44,8.88,7.79,8.67
7,435,Taman Sipung Permai,4,9.97,8.47,8.71,5.86,8.66
8,1043,Taman Banjarwangunan 3,6,8.70,8.95,8.77,7.87,8.66
9,748,Taman Banjarwangunan 1,4,9.26,8.80,8.62,7.02,8.65


### 📊 **Interpretasi Hasil**
Output menampilkan:
- **Nama Proyek dan Blok ID** → identitas unik proyek.  
- **Rata-Rata Skor per Aspek** → kualitas masing-masing dimensi proyek.  
- **Jumlah Aktivitas** → banyaknya penilaian atau survei yang masuk.  
- **Composite Quality Score** → skor total berbobot yang mencerminkan kualitas keseluruhan proyek.

Interpretasinya:
- Proyek dengan **skor di atas 8.5** menunjukkan kualitas tinggi dan kepuasan konsumen yang baik.  
- Proyek dengan **aktivitas banyak dan skor tinggi** merupakan **proyek unggulan**.  
- Jika proyek memiliki **aktivitas sedikit namun skor tinggi**, maka bisa menjadi **proyek potensial** yang layak untuk promosi lebih lanjut.  
- Sebaliknya, proyek dengan **aktivitas tinggi tetapi skor rendah** perlu **evaluasi kualitas fisik dan infrastruktur**.

### ✅ **Kesimpulan**
Analisis **Composite Quality Score per Blok Proyek** membantu perusahaan untuk:
- **Mengidentifikasi proyek dengan kualitas tertinggi** berdasarkan persepsi konsumen.  
- **Menentukan prioritas promosi dan peningkatan mutu** berdasarkan data objektif.  
- **Mendukung pengambilan keputusan manajerial** terkait pengembangan proyek baru atau perbaikan proyek yang sudah ada.  

Dengan demikian, hasil query ini menjadi alat penting untuk **monitoring kualitas proyek secara menyeluruh dan terukur.**

## 🏆 **Query 9: Identifikasi Staff Marketing Terbaik per Blok Proyek (Multi-level CTE)**

### 🎯 **Tujuan**
Menentukan **staff marketing dengan performa terbaik pada setiap blok proyek**, berdasarkan kombinasi dari **tingkat konversi prospek, skor rata-rata prospek, dan kualitas blok proyek**.  
Analisis ini membantu mengidentifikasi staff yang paling efektif dalam menghasilkan prospek berkualitas tinggi di proyek tertentu.

In [22]:
query = """

-- Query 9: Staff terbaik per blok (multi-level CTE)
WITH staff_block AS (
  SELECT
    a.staff_marketing_id,
    a.blok_id,
    COUNT(a.activity_id) AS aktivitas_staff_block,
    SUM(CASE WHEN a.prospek_konsumen = 'tertarik' THEN 1 ELSE 0 END) AS tertarik_staff_block,
    ROUND(AVG(a.skor_prospek_konsumen),2) AS avg_skor_prospek_staff
  FROM aktivitas a
  GROUP BY a.staff_marketing_id, a.blok_id
),
block_quality AS (
  SELECT
    blok_id,
    ROUND(AVG(skor_kualitas_bangunan),2) AS avg_kualitas_bangunan
  FROM aktivitas
  GROUP BY blok_id
),
staff_rank AS (
  SELECT
    sb.staff_marketing_id,
    sb.blok_id,
    sb.aktivitas_staff_block,
    sb.tertarik_staff_block,
    sb.avg_skor_prospek_staff,
    bq.avg_kualitas_bangunan,
    ROUND(
      COALESCE((CAST(sb.tertarik_staff_block AS REAL)/sb.aktivitas_staff_block)*100,0)*0.6
      + COALESCE(sb.avg_skor_prospek_staff,0)*0.3
      + COALESCE(bq.avg_kualitas_bangunan,0)*0.1
    ,2) AS staff_block_score
  FROM staff_block sb
  LEFT JOIN block_quality bq ON sb.blok_id = bq.blok_id
  WHERE sb.aktivitas_staff_block > 2
)
SELECT
  sr.blok_id,
  p.nama_proyek,
  sr.staff_marketing_id,
  s.nama_marketing,
  sr.aktivitas_staff_block,
  sr.tertarik_staff_block,
  sr.avg_skor_prospek_staff,
  sr.staff_block_score
FROM staff_rank sr
LEFT JOIN staff_marketing s ON sr.staff_marketing_id = s.staff_marketing_id
LEFT JOIN proyek p ON sr.blok_id = p.blok_id
ORDER BY sr.blok_id, sr.staff_block_score DESC;


"""

df = pd.read_sql(query, engine)
df

,blok_id,nama_proyek,staff_marketing_id,nama_marketing,aktivitas_staff_block,tertarik_staff_block,avg_skor_prospek_staff,staff_block_score
0,33,Green Watubelah,26,Farel Firmansyah,3,0,4.00,2.00
1,47,Green Watubelah,18,Basuki Rachmat Wijaya,3,0,4.67,2.16
2,114,Green Astapada,22,Kurniawan Wicaksono,3,0,6.00,2.59
3,119,Green Astapada,22,Kurniawan Wicaksono,3,0,2.67,1.62
4,124,Green Astapada,44,Bagas Pambudi Rahmat,3,0,4.67,2.11
...,...,...,...,...,...,...,...,...
212,2740,Gebang Permai,249,Asep Azis Saprizal,3,0,4.67,2.16
213,2743,Gebang Permai,250,Susilawati,3,0,4.67,2.24
214,2752,Gebang Permai,249,Asep Azis Saprizal,3,0,4.00,1.86
215,2852,Gebang Permai,238,Yeni Aprilia,5,0,4.40,1.97


### 📊 **Interpretasi Hasil**
Output menampilkan **daftar staff terbaik di setiap blok proyek**, dengan metrik performa terukur:
- **Staff_block_score tinggi** → menunjukkan kombinasi konversi tinggi dan kualitas prospek yang baik.  
- **Perbandingan antar staff di blok yang sama** membantu melihat siapa yang paling efektif di lingkungan proyek yang sama.  
- **Blok dengan rata-rata kualitas tinggi** biasanya mendukung performa staff yang lebih baik.

Interpretasinya:
- Jika **satu staff secara konsisten menempati posisi teratas di berbagai blok**, berarti ia memiliki **kemampuan marketing unggul dan adaptif**.  
- Jika **blok tertentu selalu menghasilkan skor tinggi**, bisa jadi karena **dukungan kualitas proyek yang baik**, bukan hanya performa staff.  
- Sebaliknya, jika skor blok rendah tapi ada staff dengan performa tinggi, berarti **kemampuan personal marketing staff sangat menonjol**.


### ✅ **Kesimpulan**
Analisis ini memberikan insight mendalam untuk:
- **Menentukan staff terbaik per blok proyek** secara objektif berbasis data.  
- **Mengidentifikasi blok proyek dengan potensi penjualan terbaik.**  
- **Memberikan dasar untuk sistem reward & performance evaluation** yang lebih adil dan berbasis hasil nyata.

Dengan demikian, hasil dari query ini menjadi **alat evaluasi performa individual marketing dan kualitas proyek** dalam konteks yang saling terhubung.

---

# 🧾 **Penutup**

## 🎯 **Ringkasan Akhir**
Seluruh analisis SQL yang telah dilakukan memberikan gambaran menyeluruh tentang **aktivitas marketing, kualitas proyek, dan potensi wilayah konsumen**.  
Beberapa poin penting yang dapat disimpulkan:

1. **Efektivitas Marketing Staff & Supervisor**  
   - Query 4–5 menampilkan performa individual berdasarkan hasil prospek dan keterlibatan di lapangan.  
   - Dapat digunakan untuk menilai produktivitas serta bahan evaluasi insentif atau pelatihan.

2. **Faktor Penentu Kualitas Prospek**  
   - Dari Query 6 dan 7, dapat terlihat bahwa *sumber referensi informasi* dan *wilayah konsumen* berperan besar terhadap skor prospek rata-rata.  
   - Hasil ini dapat menjadi dasar untuk **optimalisasi strategi promosi** di channel paling efektif.

3. **Kualitas Proyek dan Akses Lokasi**  
   - Query 8 membentuk metrik gabungan (*Composite Quality Score*) yang membantu mengukur **kualitas keseluruhan proyek (blok)** dengan bobot yang proporsional antar aspek.  
   - Data ini penting untuk mendukung **peningkatan mutu proyek perumahan**.

4. **Identifikasi Staff Terbaik per Blok**  
   - Query 9 menggunakan teknik **CTE (Common Table Expression)** untuk melakukan peringkat kinerja marketing secara detail di setiap blok proyek.  
   - Skor akhir dapat digunakan sebagai dasar **penghargaan kinerja (reward system)** dan alokasi tenaga kerja strategis.

---

## 🧠 **Insight Strategis**
Analisis ini membuka peluang untuk langkah lanjutan seperti:

- 📍 **Segmentasi Wilayah dan Target Pasar**:  
  Menggabungkan hasil performa kecamatan dan skor prospek untuk menentukan area dengan potensi penjualan tertinggi.

- 💼 **Performance Dashboard**:  
  Mengonversi hasil query ke dalam visualisasi interaktif (misal Power BI atau Python/Streamlit) agar pimpinan dapat memantau metrik utama secara real-time.

- 🧩 **Integrasi Machine Learning**:  
  Dataset hasil analisis dapat digunakan sebagai *training data* untuk memprediksi peluang ketertarikan prospek atau efektivitas channel promosi tertentu.

---

## 🛠️ **Rekomendasi Pengembangan**
1. Tambahkan **dimensi waktu (bulan, kuartal, tahun)** untuk memantau tren aktivitas dan performa marketing.  
2. Lakukan **data enrichment** dengan menggabungkan data eksternal (misalnya peta wilayah, data BPS, atau data pembiayaan KPR).  
3. Bangun **laporan otomatis (scheduled report)** menggunakan Python atau sistem BI yang terintegrasi langsung dengan database.

---

## ✍️ **Penutup Akhir**
Analisis SQL ini tidak hanya menjadi alat eksplorasi data, tetapi juga menjadi **fondasi sistem evaluasi kinerja berbasis data (data-driven decision-making)**.  
Dengan pendekatan ini, perusahaan dapat:
- Meningkatkan efektivitas strategi marketing,  
- Mengoptimalkan pengembangan proyek, dan  
- Meningkatkan pengalaman serta kepuasan calon konsumen.

---

📅 **Tanggal Analisis:** 27 September 2025

👤 **Disusun oleh:** Feby Renaldi

🏢 **Divisi:** Data Analyst PT Karya Berkah Gemilang

---

> *"Data yang baik tidak hanya menjelaskan masa lalu, tetapi juga menuntun arah masa depan."*  
> — **Laporan Analisis SQL Aktivitas Marketing & Proyek Perumahan**